### Import Libraries

In [ ]:
!pip install selenium
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 2.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import requests
import re
import time
import datetime
import json
import math
import unidecode
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup

In [ ]:
url_user_TV365 = 'https://timviec365.vn/nguoi-tim-viec.html'

In [ ]:
user_TV365 = requests.get(url_user_TV365).content
print(user_TV365)

b'<!DOCTYPE html>\r\n<html lang="en">\r\n\r\n<head>\r\n  <meta charset="UTF-8">\r\n  <meta name="viewport" content="width=device-width, initial-scale=1.0">\r\n  <title>\r\n    T\xc3\xacm \xe1\xbb\xa8ng Vi\xc3\xaan Mi\xe1\xbb\x85n Ph\xc3\xad, Danh S\xc3\xa1ch H\xe1\xbb\x93 S\xc6\xa1 \xe1\xbb\xa8ng Vi\xc3\xaan Mi\xe1\xbb\x85n Ph\xc3\xad 05/2025\r\n  </title>\r\n  <meta name="robots" content="index,follow" />\r\n  <meta name="title" content="T\xc3\xacm \xe1\xbb\xa8ng Vi\xc3\xaan Mi\xe1\xbb\x85n Ph\xc3\xad, Danh S\xc3\xa1ch H\xe1\xbb\x93 S\xc6\xa1 \xe1\xbb\xa8ng Vi\xc3\xaan Mi\xe1\xbb\x85n Ph\xc3\xad 05/2025" />\r\n  <meta name="description" content="T\xc3\xacm \xe1\xbb\xa9ng vi\xc3\xaan mi\xe1\xbb\x85n ph\xc3\xad nh\xc6\xb0 th\xe1\xba\xbf n\xc3\xa0o? T\xc3\xacm CV mi\xe1\xbb\x85n ph\xc3\xad \xe1\xbb\x9f \xc4\x91\xc3\xa2u? S\xc4\x83n t\xc3\xacm h\xe1\xbb\x93 s\xc6\xa1 \xe1\xbb\xa9ng vi\xc3\xaan ng\xc6\xb0\xe1\xbb\x9di t\xc3\xacm vi\xe1\xbb\x87c cho c\xc3\xa1c v\xe1\xbb\x8b tr\xc3\xad hot t

In [ ]:
soup_user_TV365 = BeautifulSoup(user_TV365, 'html.parser')
print(soup_user_TV365)

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>
    Tìm Ứng Viên Miễn Phí, Danh Sách Hồ Sơ Ứng Viên Miễn Phí 05/2025
  </title>
<meta content="index,follow" name="robots">
<meta content="Tìm Ứng Viên Miễn Phí, Danh Sách Hồ Sơ Ứng Viên Miễn Phí 05/2025" name="title">
<meta content="Tìm ứng viên miễn phí như thế nào? Tìm CV miễn phí ở đâu? Săn tìm hồ sơ ứng viên người tìm việc cho các vị trí hot trực tuyến như thế nào? Khám phá ngay!" name="description"/>
<link href="https://timviec365.vn/nguoi-tim-viec.html" rel="canonical"/>
<link href="https://fonts.googleapis.com" rel="preconnect"/>
<link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/>
<!-- <link rel="preload" as="font"
    href="https://fonts.googleapis.com/css2?family=Roboto:ital,wght@0,400;0,500;0,700;1,400;1,700&display=swap"> -->
<meta content="summary" name="twitter:card"/>
<meta content="@timviec365vn" name="twit

### Crawl url page

In [1]:
# Viết hàm lấy url page chứa danh sách các công việc
def get_url_page_user_TV365(num_page):
  list_url_page = []
  for i in range(1,num_page+1):
    list_url_page.append('https://timviec365.vn/nguoi-tim-viec.html?page='+str(i))
  return list_url_page

In [ ]:
# Lấy danh sách các page có danh sách các công việc
# last_page = soup_user_TV365.find('a', class_=' last')['href']
total_pages = 5000
list_url_page = get_url_page_user_TV365(total_pages)
list_url_page

['https://timviec365.vn/nguoi-tim-viec.html?page=1',
 'https://timviec365.vn/nguoi-tim-viec.html?page=2',
 'https://timviec365.vn/nguoi-tim-viec.html?page=3',
 'https://timviec365.vn/nguoi-tim-viec.html?page=4',
 'https://timviec365.vn/nguoi-tim-viec.html?page=5',
 'https://timviec365.vn/nguoi-tim-viec.html?page=6',
 'https://timviec365.vn/nguoi-tim-viec.html?page=7',
 'https://timviec365.vn/nguoi-tim-viec.html?page=8',
 'https://timviec365.vn/nguoi-tim-viec.html?page=9',
 'https://timviec365.vn/nguoi-tim-viec.html?page=10',
 'https://timviec365.vn/nguoi-tim-viec.html?page=11',
 'https://timviec365.vn/nguoi-tim-viec.html?page=12',
 'https://timviec365.vn/nguoi-tim-viec.html?page=13',
 'https://timviec365.vn/nguoi-tim-viec.html?page=14',
 'https://timviec365.vn/nguoi-tim-viec.html?page=15',
 'https://timviec365.vn/nguoi-tim-viec.html?page=16',
 'https://timviec365.vn/nguoi-tim-viec.html?page=17',
 'https://timviec365.vn/nguoi-tim-viec.html?page=18',
 'https://timviec365.vn/nguoi-tim-vie

### Crawl url user

In [ ]:
def slugify(name):
    # Bỏ dấu tiếng Việt và tạo slug
    name = unidecode.unidecode(name).lower()
    name = re.sub(r'[^a-z0-9\s-]', '', name)
    name = re.sub(r'\s+', '-', name)
    return name.strip('-')

def get_url_user_TV365(url_page):
    list_url_user = []

    try:
        # Lấy nội dung trang
        response = requests.get(url_page)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Tìm <script id="__NEXT_DATA__"> chứa JSON
        script_tag = soup.find('script', id="__NEXT_DATA__")
        if script_tag:
            data = json.loads(script_tag.string)
            user_list = data['props']['pageProps']['list_uv_new']

            for user in user_list:
                name = user.get('use_first_name', '')
                user_id = user.get('use_id')
                if name and user_id:
                    slug = slugify(name)
                    url = f"https://timviec365.vn/uvtv2/{slug}-uv{user_id}"
                    list_url_user.append(url)

    except Exception as e:
        print(f"Error: {e}")

    return list_url_user

In [ ]:
list_url_user_TV365 = []
for url_page in tqdm(list_url_page):
  try:
    list_url_user_TV365.extend(get_url_user_TV365(url_page))
  except:
    print(f'{url_page} -> Error')

len(list_url_user_TV365)

  4%|▎         | 177/5000 [04:12<25:25:32, 18.98s/it]

Error: 504 Server Error: Gateway Time-out for url: https://timviec365.vn/search-ung-vien?page=177


  4%|▍         | 219/5000 [06:00<25:16:11, 19.03s/it]

Error: 504 Server Error: Gateway Time-out for url: https://timviec365.vn/search-ung-vien?page=219


 13%|█▎        | 660/5000 [16:08<1:52:47,  1.56s/it]

...

Error: 'NoneType' object is not iterable


 99%|█████████▉| 4971/5000 [3:16:52<01:09,  2.39s/it]

Error: 'NoneType' object is not iterable


100%|██████████| 5000/5000 [3:18:42<00:00,  2.38s/it]


99260

In [ ]:
list_url_user_TV365 = list(set(list_url_user_TV365))
len(list_url_user_TV365)

98760

In [ ]:
df_url_user_TV365 = pd.DataFrame({'URL User': list_url_user_TV365})
df_url_user_TV365

,URL User
0,https://timviec365.vn/uvtv2/nguyen-ngoc-ninh-u...
1,https://timviec365.vn/uvtv2/nguyen-thi-ha-uv11...
2,https://timviec365.vn/uvtv2/vnbnvcvn-uv195365
3,https://timviec365.vn/uvtv2/nguyen-thi-nhat-ph...
4,https://timviec365.vn/uvtv2/truong-hoang-nhi-u...
...,...
98755,https://timviec365.vn/uvtv2/ruan-shi-jin-yin-u...
98756,https://timviec365.vn/uvtv2/nguyen-thi-trang-u...
98757,https://timviec365.vn/uvtv2/vo-quang-quy-uv336199
98758,https://timviec365.vn/uvtv2/tran-van-loc-uv111...


In [ ]:
df_url_user_TV365.to_csv('user_url_TV365.csv',index_label='ID')

### Crawl user data

In [ ]:
def normalize_text(text):
  text = re.sub('\n+', ' ', text)
  text = re.sub('\r', '', text)
  text = re.sub('   ', '', text)
  text = re.sub('  ', ' ', text)
  return text

In [ ]:
# Khởi tạo driver object để chạy hàm cào data ở dưới, trong trường hợp cào data bị dừng đột ngột thì chạy lại cell này để khởi
# tạo lại driver
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--window-size=1920,1080")

service = Service()
driver = webdriver.Chrome(service=service, options=chrome_options)

In [ ]:
def extract_candidate_details(html):
    """
    Extract candidate details from the provided HTML. using Selenium
        Ngành nghề: 'industry',
        Nơi mong muốn làm việc: 'workplace_desired',
        Quận/huyện: 'district',
        Phường/Xã: 'current_place_of_residence',
        Quận huyện - Tỉnh thành': 'current_place_of_residence',
        Mức lương mong muốn: 'desired_salary',
        Kinh nghiệm làm việc: 'work_experience',
        Hình thức làm việc: 'employment_type',
        Cấp bậc: 'position_level',
        Tuổi: 'age',
        Giới tính: 'gender',
    """


    soup = BeautifulSoup(html, 'html.parser')
    container = soup.find('div', class_='canidate_detail_container_main__LjREw')
    if not container:
        print("Target div not found.")
        return None

    # --- Get desired job title ---
    desired_job_tag = container.find('h1')
    desired_job = desired_job_tag.get_text(strip=True) if desired_job_tag else ''

    # --- Get user ID and view count ---
    user_info = {
        'userid': None,
        'view_count': None
    }

    p_tags = container.find_all('p')
    for p in p_tags:
        text = p.get_text(strip=True)
        if 'Mã ứng viên:' in text:
            span = p.find('span')
            user_info['userid'] = span.get_text(strip=True) if span else text.split(':')[-1].strip()
        elif 'Lượt mở:' in text:
            span = p.find('span')
            user_info['view_count'] = span.get_text(strip=True) if span else text.split(':')[-1].strip()

    # --- Mapping label to output key ---
    label_map = {
        'Ngành nghề': 'industry',
        'Nơi mong muốn làm việc': 'workplace_desired',
        'Quận/huyện': 'district',
        'Phường/Xã': 'current_place_of_residence',
        'Quận huyện - Tỉnh thành': 'current_place_of_residence',
        'Mức lương mong muốn': 'desired_salary',
        'Kinh nghiệm làm việc': 'work_experience',
        'Hình thức làm việc': 'employment_type',
        'Cấp bậc': 'position_level',
        'Tuổi': 'age',
        'Giới tính': 'gender',
    }

    # --- Extract labeled data ---
    data = {}
    for p in container.find_all('p'):
        label = p.get_text(strip=True)
        for key, field_name in label_map.items():
            if key in label:
                # Try to find <a> or <span> inside the <p> tag for the value
                value_tag = p.find('a') or p.find('span')
                value = value_tag.get_text(strip=True) if value_tag else label.split(':')[-1].strip()
                data[field_name] = value
                break

    # Fill defaults for missing fields
    for field in label_map.values():
        data.setdefault(field, None)

    result = {
        'desired_job': desired_job,
        'userid': user_info['userid'],
        'user_name': '',  # To be extracted differently if needed
        'industry': data['industry'],
        'workplace_desired': data['workplace_desired'],
        'province': data['workplace_desired'],  # Same as workplace_desired
        'district': data['district'],
        'current_place_of_residence': data['current_place_of_residence'],
        'desired_salary': data['desired_salary'],
        'work_experience': data['work_experience'],
        'gender': data['gender'],
        'age': data['age'],
        'career_goals': '',  # Extracted separately
        'skills': '',        # Extracted separately
        'marriage': '',      # Extracted separately
    }

    return result


In [ ]:
def extract_json_data_fields(html):
    """
    Extract name, career_goals, skills, and marriage from __NEXT_DATA__ script tag including

       Tên ứng viên: 'user_name':
       Mục tiêu công việc: 'career_goals',
       Kỹ năng: 'skills' ,
       Tình trạng hôn nhân: 'marriage'

    """
    try:
        soup = BeautifulSoup(html, 'html.parser')

        # Find the JSON data in the __NEXT_DATA__ script tag
        script_tag = soup.find('script', {'id': '__NEXT_DATA__'})
        if not script_tag:
            raise Exception("Could not find __NEXT_DATA__ script tag")

        # Parse JSON data
        json_data = json.loads(script_tag.string)
        thong_tin = json_data['props']['pageProps']['in4']['thong_tin']

        # Helper function to safely get values
        def safe_get(data, key, default='?'):
            value = data.get(key, default)
            return str(value) if value is not None and value != '' else default

        # Extract marriage status
        marriage_status = thong_tin.get('use_hon_nhan')
        if marriage_status == 0:
            marriage = 'Độc thân'
        elif marriage_status == 1:
            marriage = 'Đã kết hôn'
        else:
            marriage = '?'

        return {
            'user_name': safe_get(thong_tin, 'use_first_name'),
            'career_goals': safe_get(thong_tin, 'cv_muctieu'),
            'skills': safe_get(thong_tin, 'cv_kynang'),
            'marriage': marriage
        }

    except Exception as e:
        print(f"Error extracting JSON data: {str(e)}")
        return {
            'user_name': '?',
            'career_goals': '?',
            'skills': '?',
            'marriage': '?'
        }


In [ ]:
def crawl_user_data_TV365_combined(url, driver):
    """
    Combined function that uses extract_candidate_details for most fields (via Selenium)
    and extracts name, career_goals, skills, marriage from __NEXT_DATA__ (via requests)

    Đối số:
        url: The URL to scrape
        driver: Selenium WebDriver instance (already initialized)
    """
    try:
        # Get rendered HTML through Selenium for extract_candidate_details
        driver.get(url)
        # You might need to add wait time or wait for specific elements to load
        time.sleep(2)  # or use WebDriverWait
        selenium_html = driver.page_source

        # Extract data from Selenium-rendered HTML using extract_candidate_details
        html_data = extract_candidate_details(selenium_html)
        if not html_data:
            raise Exception("Could not extract candidate details from Selenium HTML")

        # Get initial HTML via requests for __NEXT_DATA__ (faster than Selenium)
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Extract additional fields from JSON data in initial HTML
        json_data = extract_json_data_fields(response.content)

        # Combine both data sources
        combined_result = {
            'user_url': url,
            'userid': html_data['userid'],
            'user_name': json_data['user_name'],  # From JSON
            'industry': [html_data['industry']] if html_data['industry'] else [],
            'desired_job': html_data['desired_job'],
            'workplace_desired': [html_data['workplace_desired']] if html_data['workplace_desired'] else [],
            'current_place_of_residence': html_data['current_place_of_residence'],
            'province': html_data['province'],
            'district': html_data['district'],
            'desired_salary': html_data['desired_salary'],
            'gender': html_data['gender'],
            'marriage': json_data['marriage'],  # From JSON
            'date_of_birth': html_data.get('age', '?'),  # Using age from HTML
            'work_experience': html_data['work_experience'],
            'career_goals': json_data['career_goals'],  # From JSON
            'skills': json_data['skills'],  # From JSON
            'employment_type': html_data.get('employment_type', '?'),
            'position_level': html_data.get('position_level', '?'),
            'view_count': html_data.get('view_count', '?')
        }

        # Return as list format to match original function
        return [
            combined_result['user_url'],         # 0
            combined_result['userid'],           # 0
            combined_result['user_name'],        # 1
            combined_result['industry'],         # 2
            combined_result['desired_job'],      # 3
            combined_result['workplace_desired'], # 4
            combined_result['current_place_of_residence'], # 5
            combined_result['province'],         # 6
            combined_result['district'],         # 7
            combined_result['desired_salary'],   # 8
            combined_result['gender'],           # 9
            combined_result['marriage'],         # 10
            combined_result['date_of_birth'],    # 11
            combined_result['work_experience'],  # 12
            combined_result['career_goals'],     # 13
            combined_result['skills']            # 14
        ]

    except Exception as e:
        print(f"Error crawling {url}: {str(e)}")
        return None

In [ ]:
# Lấy dữ liệu chứa đường dẫn tới ứng viên
url = 'https://drive.google.com/file/d/1tzLoNTBCOh5gwYuKVwI4bGNdYFz3sNrM/view?usp=sharing'
file_id = url.split('/d/')[1].split('/')[0]
csv_url = f'https://drive.google.com/uc?id={file_id}'
df_url = pd.read_csv(csv_url)
df_url.head()

,ID,URL User
0,0,https://timviec365.vn/uvtv2/nguyen-cam-tien-uv...
1,1,https://timviec365.vn/uvtv2/pham-nguyen-phuc-d...
2,2,https://timviec365.vn/uvtv2/nguyen-thai-tuan-u...
3,3,https://timviec365.vn/uvtv2/bui-thuy-van-nhan-...
4,4,https://timviec365.vn/uvtv2/tran-thi-khen-uv11...


In [ ]:
# Thay đổi code chỗ này
urls = df_url["URL User"][20000:23000]

# Define the column names
columns = [
    "user_url", "userid", "user_name", "industry", "desired_job", "workplace_desired",
    "current_place_of_residence", "province", "district", "desired_salary",
    "gender", "marriage", "age", "work_experience", "career_goals", "skills"
]

# Store results
data = []

In [ ]:
#Chạy cell này để bắt đầu cào data
for url in urls:
    try:
        combined_result = crawl_user_data_TV365_combined(url, driver)

        # Make sure result has all 16 items
        if combined_result and len(combined_result) == 16:
            row = combined_result  # already a list in correct order
            data.append(row)
        else:
            print(f"Skipped {url}: unexpected result length.")
    except Exception as e:
        print(f"Error processing {url}: {e}")


Target div not found.
Error crawling https://timviec365.vn/uvtv2/nguyen-tuan-anh-uv1111679359: Could not extract candidate details from Selenium HTML
Skipped https://timviec365.vn/uvtv2/nguyen-tuan-anh-uv1111679359: unexpected result length.
Error crawling https://timviec365.vn/uvtv2/nguyen-anh-tu-uv215657: 504 Server Error: Gateway Time-out for url: https://timviec365.vn/uvtv2/nguyen-anh-tu-uv215657
Skipped https://timviec365.vn/uvtv2/nguyen-anh-tu-uv215657: unexpected result length.


In [ ]:
# Create DataFrame
result_df = pd.DataFrame(data, columns=columns)

# Show result
result_df

In [ ]:
result_df.to_csv('user_data_TV365.csv', index = False)